# Decision Tree Learning: Prosper Loan Dataset

A decision tree a learned set of rules that allows us to make decisions on data.

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

In [ ]:
%matplotlib inline
import time
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

## Step 1: Load the Data

In [ ]:
## small file, start with this
datafile = "/data/prosper-loan/prosper-loan-data-sample.csv"
## this is a large file
#datafile = "/data/prosper-loan/prosper-loan-data.csv.gz"

t1 = time.perf_counter()
data = spark.read. \
          option("header", "true"). \
          option("inferSchema", "true").  \
          csv(datafile)
t2 = time.perf_counter()

print("read {:,} records in {:,.2f} ms".format(data.count(), (t2-t1)*1000))
# schema
data.printSchema()

In [ ]:
## TODO : select a few columns 
## start with: 'LoanStatus',  'EmploymentStatus', 'CreditScore', 'StatedMonthlyIncome', 'ListingCategory'
## we can add more later

select_columns = ['LoanStatus', 'EmploymentStatus', 'CreditScore', '???', '???']

## Note : vector columns can only have Numbers, don't include Categorical columns here
## And dfefinitely not 'LoanStatus'  (if you are curiuos include and see what happens!)
vector_columns = [ 'EmpIndex', 'CreditScore', 'StatedMonthlyIncome', 'CategoryIndex']



In [ ]:
## TODO : Extract only the columns we are interested in
## Hint : 'select_columns'
prosper = data.select(???)  

print (prosper.count())
prosper.show(10)

## Step 2 : Clean Data

In [ ]:
## TODO :  Drop any NA, null values.  
## Hint : Using `.na.drop()`
prosper_clean = prosper.na.???()

print("Original record count {:,}, cleaned records count {:,},  dropped {:,}"\
      .format(prosper.count(), prosper_clean.count(), (prosper.count() - prosper_clean.count())))
prosper_clean.show()


## Look at some summary data

**=> Q : What does that say about the cardinality of these categorical columns?**


In [ ]:
## TODO : Look at some summaries
## We are going to group counts by 'LoanStatus',  'EmploymentStatus' ,   'ListingCategory'

prosper_clean.groupBy('LoanStatus').count().show()
prosper_clean.groupBy('EmploymentStatus').count().show()
prosper_clean.groupBy('???').count().show(60)

## Question : What does that say about the cardinality of these categorical columns? *

## Step 3: Converting Categorical columns 

Convert categorical columns to numeric.   
Here let's convert **EmploymentStatus** column

In [ ]:
## TODO : Create a StringIndexer with inputCol='EmploymentStatus'  and outputCol = 'EmpIndex'
strIndexer_employment = StringIndexer(inputCol="???", outputCol="???")
prosper_indexed = strIndexer_employment.fit(prosper_clean).transform(prosper_clean)

prosper_indexed.show(10)


In [ ]:
## TODO : Create another StringIndexer,  inputCol='ListingCategory',   outputCol='CategoryIndex'
strIndexer_category = StringIndexer(inputCol="ListingCategory", outputCol="CategoryIndex")
prosper_indexed = strIndexer_category.fit(prosper_indexed).transform(prosper_indexed)

prosper_indexed.show()


## Step 4: Build feature vectors using VectorAssembler.

In [ ]:
assembler = VectorAssembler(inputCols=vector_columns, outputCol="features")
feature_vector = assembler.transform(prosper_indexed)
feature_vector = feature_vector.withColumn("label", feature_vector["LoanStatus"])
feature_vector.show(10, False)

## Step 5: Split Data into training and test.

We will split our the data up into training and test.  (You know the drill by now).

**=> TODO: Split dataset into 70% training, 30% validation**


In [ ]:
## TODO :  Split the data into 70% training and 30% test sets 
## Hint : 0.7   , 0.3
(training, test) =  feature_vector.randomSplit([???,  ???])
print("training set = " , training.count())
print("testing set = " , test.count())

## Step 6: Decision Tree
** Q : How many nodes the tree has? **

### 6.1 Create Tree

In [ ]:
##  TODO: Create a DecisionTree model with 5000 Maxbins
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=???)

### 6.2 Train Tree

In [ ]:
t1 = time.perf_counter()
## TODO : train with training data set
## Hint : training
tree_model = dt.fit(???)
t2 = time.perf_counter()

## TODO : Notice the time took to train
print("traind on {:,} records using {:,} features in {:,.2f} ms".\
      format(training.count(), len(vector_columns), (t2-t1)*1000))

### 6.3 Print Tree

In [ ]:
## TODO : Observe the output
## how many nodes does the tree have?
print(tree_model)
print()
print(tree_model.toDebugString)

### 6.4 Predict

In [ ]:
## TODO : create predictions using test dataset
## Hint : test
predictions = tree_model.transform(???)

predictions2= predictions.drop('rawPrediction', 'probability')
predictions2.show()


## Step 7: Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("accuracy ",  accuracy)


## Step 8: Improve Accuracy

### Add more data
In Step-1 change the 'datafile' to the full dataset.  
And see how the accuracy above changes

### Add more features
Look at the schema of the full dataset.  Are there any columns you want to add